### CryoParticleSegment

In [ ]:
#%%bash
#git clone git+https://cyanazuki@github.com/cyanazuki/CryoParticleSegment.git
#cd CryoParticleSegment
#python setup.py install

## ⭐ Setup
You must run all codes under this category.

### ✅ Directory Settings

In [1]:
# @title  { display-mode: "form" }

IMAGE_DIR = "/content/drive/MyDrive/research_xs/processed_micrographs_np" # @param {type:"string"}
LABEL_DIR = "/content/drive/MyDrive/research_xs/ground_truth_mask" # @param {type:"string"}
RESULT_DIR = "/content/drive/MyDrive/research_xs/final/deeplabv3_resnet50/Crf_model_convcrf" # @param {type:"string"}

In [2]:
# @title  { display-mode: "form" }
# @markdown Detect whether using folder in Google Drive as **`RESULT DIR`**📁.

if "content" in IMAGE_DIR.split("/")[:3]:
  try:
    from google.colab import drive
    drive.mount('/content/drive')
    !rm -r /content/sample_data
  except:
    pass

Mounted at /content/drive


In [3]:
# @title  { display-mode: "form" }
# @markdown Source code directory.
SRC_DIR = "/content/drive/MyDrive/research/src" # @param {type:"string"}

if True:
  !cp -r {SRC_DIR}/EM_project/*.py /content/
else:
  !cp {SRC_DIR}/EM_project/convcrf.py /content/convcrf.py
  !cp {SRC_DIR}/EM_project/dataset.py /content/dataset.py
  !cp {SRC_DIR}/EM_project/lr_scheduler.py /content/lr_scheduler.py
  !cp {SRC_DIR}/EM_project/metrics.py /content/metrics.py
  !cp {SRC_DIR}/EM_project/model.py /content/model.py
  !cp {SRC_DIR}/EM_project/trainer.py /content/trainer.py
  !cp {SRC_DIR}/EM_project/utils.py /content/utils.py

### ✅ Packages Handling

In [4]:
# @title  { display-mode: "form" }
# @markdown Useful packages.

import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [5]:
# @title  { display-mode: "form" }
# @markdown User-defined packages.

from dataset import MicrographDataset, MicrographDatasetEvery
from model import create_model
from dataset import reconstruct_patched
from trainer import CryoEMEvaluator, tqdm_plugin_for_Trainer

## ⭐ Main

### ✅ Setting

In [6]:
# @markdown Parameters.

NUM_CLASSES = 2
EPOCHS = 300
BATCH = 2
CROP_SIZE = (1024, 1024)
LR = 1e-5
RLR_PATIENCE = 3
ES_PATIENCE = 20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# @markdown Set seed.

random_state = 42
torch.manual_seed(random_state)
torch.cuda.manual_seed_all(random_state)

### ✅ Dataset

In [8]:
train_dir = os.path.join(IMAGE_DIR, 'train')
train_filenames = np.loadtxt(f"{IMAGE_DIR}/train_filenames.txt", dtype=str)
train_dataset = MicrographDataset(image_dir=train_dir, label_dir=LABEL_DIR, filenames=train_filenames, crop_size=CROP_SIZE)

In [9]:
val_dir = os.path.join(IMAGE_DIR, 'val')
val_filenames = np.loadtxt(f"{IMAGE_DIR}/val_filenames.txt", dtype=str)
val_dataset = MicrographDatasetEvery(image_dir=val_dir, label_dir=LABEL_DIR, filenames=val_filenames, crop_size=CROP_SIZE)
val_loader = DataLoader(val_dataset, batch_size=None, shuffle=False, pin_memory=True)

In [10]:
test_dir = os.path.join(IMAGE_DIR, 'test')
test_filenames = np.loadtxt(f"{IMAGE_DIR}/test_filenames.txt", dtype=str)
test_dataset = MicrographDatasetEvery(image_dir=test_dir, label_dir=LABEL_DIR, filenames=test_filenames, crop_size=CROP_SIZE)
test_loader = DataLoader(test_dataset, batch_size=None, shuffle=False, pin_memory=True)

### ✅ Model

In [11]:
backbone = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)

backbone.backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
backbone.classifier[4] = torch.nn.Conv2d(256, NUM_CLASSES, kernel_size=1, stride=1)
backbone.aux_classifier[4] = torch.nn.Conv2d(256, NUM_CLASSES, kernel_size=1, stride=1)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:00<00:00, 186MB/s]


## ⭐ Evaluate

### ✅ Load model

In [12]:
CHECKPOINT_PATH = "/content/drive/MyDrive/research_xs/final/deeplabv3_resnet50/checkpoint26_B2_L-3_C1024.pt" # @param {type:"string"}
state_dict_path = CHECKPOINT_PATH
state_dict = torch.load(state_dict_path, map_location=torch.device(DEVICE))
backbone.load_state_dict(state_dict, strict=False)
model = create_model(backbone)
model.to(DEVICE)
model.eval()
print("Load model at: ", state_dict_path)

Load model at:  /content/drive/MyDrive/research_xs/final/deeplabv3_resnet50/checkpoint26_B2_L-3_C1024.pt


In [13]:
import gc
gc.collect()
torch.cuda.empty_cache()

### ✅ Evalutor

In [14]:
Evaluator = tqdm_plugin_for_Trainer(CryoEMEvaluator)
evaluator = Evaluator(model=model, device=DEVICE, metrics=['iou'], num_classes=NUM_CLASSES)
print("FCN validation result:")
result = evaluator.evaluate(loader=val_loader)
print("FCN test result:")
result = evaluator.evaluate(loader=test_loader)

FCN validation result:


Validation:   0%|          | 0/9 [00:00<?, ?it/s]

global correct: 90.85
average row correct: ['96.46', '73.68']
Recall: ['91.80', '87.20']
IoU: ['88.82', '66.49']
F1 Score: ['94.08', '79.87']
mean IoU: 77.65
FCN test result:


Validation:   0%|          | 0/17 [00:00<?, ?it/s]

global correct: 90.09
average row correct: ['95.14', '72.54']
Recall: ['92.32', '81.15']
IoU: ['88.16', '62.08']
F1 Score: ['93.71', '76.61']
mean IoU: 75.12


### ✅ Evaluate convcrf

In [ ]:
# @markdown config dondong
import yaml

!cp {SRC_DIR}/config.yml /content/config.yml
config_path = "/content/config.yml"

with open(config_path, 'r') as f:
    data = yaml.safe_load(f)
    crf_args = data['crf_config']

In [15]:
# @markdown config
config = {
    'filter_size': 7,
    'blur': 4,
    'merge': True,
    'norm': 'none', # sym
    'trainable': False,
    'convcomp': False,

    'weight': 'vector', # scalar
    'unary_weight': 1,
    'weight_init': 0.2,
    'logsoftmax': True,
    'softmax': True,
    'final_softmax': False,

    'pos_feats': {
        'sdims': 3, # 3
        'compat': 3, # 3,
    },

    'col_feats': {
        'sdims': 3, # 80,
        'schan': 0.5, # 13
        'compat': 1, # 10
        'use_bias': False, # True
    },
    "trainable_bias": False,
    "pyinn": False
}

In [16]:
from model import create_crf_model

model = create_crf_model(
    backbone, config=config, shape=CROP_SIZE,
    num_classes=NUM_CLASSES, use_gpu=torch.cuda.is_available()) #config=crf_args

In [17]:
evaluator = Evaluator(
    model=model, device=DEVICE, metrics=['iou'],
    num_classes=NUM_CLASSES)
print("ConvCRF validation result:")
result = evaluator.evaluate(loader=val_loader)
print("ConvCRF test result:")
result = evaluator.evaluate(loader=test_loader)

ConvCRF validation result:


Validation:   0%|          | 0/9 [00:00<?, ?it/s]

global correct: 89.61
average row correct: ['97.32', '66.05']
Recall: ['89.76', '88.95']
IoU: ['87.59', '61.04']
F1 Score: ['93.38', '75.80']
mean IoU: 74.31
ConvCRF test result:


Validation:   0%|          | 0/17 [00:00<?, ?it/s]

global correct: 89.22
average row correct: ['96.14', '65.20']
Recall: ['90.55', '82.97']
IoU: ['87.38', '57.51']
F1 Score: ['93.26', '73.02']
mean IoU: 72.44


In [ ]:
from torchvision.utils import save_image
from dataset import reconstruct_patched

!mkdir {RESULT_DIR}
model.eval()
with torch.no_grad():
  for idx, (test_image, _, grid, _) in enumerate(test_dataset):
    inputs = test_image.to(DEVICE)
    outputs = model(inputs)['out']
    preds = outputs.argmax(dim=1).cpu().detach()
    filename = f"{os.path.splitext(test_dataset.filenames[idx])[0]}.png"
    pred_path = os.path.join(RESULT_DIR, filename)
    save_image(reconstruct_patched(preds, grid).float(), pred_path)